In [28]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
%cd /content/drive/MyDrive/Project2/FilterMethod

/content/drive/MyDrive/Project2/FilterMethod


In [30]:
import pandas as pd
import numpy as np
import random
import math

In [31]:
# Load dataset
df = pd.read_csv('./Data/CreditCard/CreditCard.csv')
df

# Drop the 'id' column
df.drop('id', axis=1, inplace=True)

# Separate the features and the target variable
X = df.drop('y', axis=1)  # Features
y = df['y']  # Target variable

In [32]:
#Read data from file csv
label = pd.read_csv("./Data/CreditCard/label.csv")
lb = label.to_numpy()
mi_Xk_Y = lb.reshape(-1)

mutual = pd.read_csv("./Data/CreditCard/mutual.csv")
mi_Xj_Xk = mutual.to_numpy()

mutual_con = pd.read_csv("./Data/CreditCard/mutual_con.csv")
cmi_Xj_Xk_Y = mutual_con.to_numpy()
print(cmi_Xj_Xk_Y.shape)

(30, 30)


In [33]:
# Tính giá trị của độ đo CIFE cho từng đặc trưng trong tập dữ liệu đầu vào.
CIFE = []
for i in range(X.shape[1]):
    I_Xk_Y = mi_Xk_Y[i]
    cij = 0
    for j in range(X.shape[1]):
      if i!=j:
        cij += -mi_Xj_Xk[i, j] + cmi_Xj_Xk_Y[i, j]
    CIFE.append(I_Xk_Y + cij)

Hàm `F(solution)` : tính toán giá trị hàm mục tiêu để đánh giá một `solution`

Input:
- `solution`: Một mảng (array) gồm các giá trị 0 hoặc 1, đại diện cho tập đặc trưng được chọn để đánh giá.

Process:
- Kiểm tra tổng các giá trị trong `solution`. Nếu tổng là 0 (không có đặc trưng nào được chọn), hàm trả về giá trị dương vô cùng (`math.inf`).
- Lấy ra giá trị `CIFE` của các feature không được chọn, tương ứng với các phần tử có giá trị 0 trong mảng `solution`, và lưu vào mảng `unselected_score`.
- Trả về tổng giá trị của `unselected_score` cộng với tổng giá trị của `solution`.

Output:
- Một số thực, đại diện cho giá trị hàm mục tiêu để đánh giá tập feature `solution`.
$$\sum_{X_i \in \mathcal{S} \backslash \mathcal{S^*}} \mathcal{F}(X_i) + \lambda \sum_{i=1}^n d_i$$

In [34]:
# the impact of unselected features
def F(solution):
  if sum(solution) == 0:
    return math.inf
  unselected_score = []
  for i in range(len(solution)):
     if solution[i] == 0: # if X
       unselected_score.append(CIFE[i])
  return sum(unselected_score) + ld*sum(solution)

Hàm `min_info_feature(candidate_solution)` : tìm ra một đặc trưng có giá trị CIFE nhỏ nhất trong tập solution cần loại bỏ để tạo ra removal-based neighborhood.

$S_1 = \mathcal{S^*} \backslash \{ f^*\}, f^* = \arg min_{f_i \in \mathcal{S^*}} \mathcal{F}(f_i)$

Input:
- `candidate_solution`: Một mảng (array) gồm các giá trị 0 hoặc 1

Process:
- Duyệt qua các phần tử trong `S` và kiểm tra các phần tử mà tại vị trí `index` trong `S` có giá trị bằng 1 (feature đã được chọn), so sánh giá trị `CIFE` của feature đó với so với `lowest_value`. Nếu nhỏ hơn, cập nhật giá trị của `lowest_feature` là `index` của feature và `lowest_value` là giá trị `CIFE` của feature đó.
- Trả về giá trị `lowest_feature`, đại diện cho feature có giá trị thấp nhất theo `CIFE`.

Output:
- Một giá trị số nguyên, là vị trí của feature có giá trị `CIFE` thấp nhất để loại bỏ khỏi solution.

In [35]:
# for generate removal-based neighborhood of a candidate solution
def min_info_feature(candidate_solution):
    S = candidate_solution.copy()
    lowest_feature = -math.inf
    lowest_value = math.inf
        
    for index in range(len(S)):
      if S[index] == 1:
          if CIFE[index] < lowest_value:
            lowest_feature = index
            lowest_value = CIFE[index]

    return lowest_feature

Hàm `max_info_feature(candidate_solution)` : tìm ra một đặc trưng có giá trị CIFE lớn nhất nằm ngoài tập solution cần thêm vào để tạo ra insertion-based neighborhood.

$S_2 = \mathcal{S^*} \cup \{ f^*\}, f^* = \arg max_{f_i \in \mathcal{S} \backslash \mathcal{S^*}} \mathcal{F}(f_i)$

Input:
- `candidate_solution`: Một mảng (array) gồm các giá trị 0 hoặc 1

Process:
- Duyệt qua các phần tử trong `S` và kiểm tra các phần tử mà tại vị trí `index` trong `S` có giá trị bằng 0 (feature không nằm trong tập được chọn), so sánh giá trị `CIFE` của feature đó với so với `highest_value`. Nếu lớn hơn, cập nhật giá trị của `highest_feature` là `index` của feature và `highest_value` là giá trị `CIFE` của feature đó.
- Trả về giá trị `highest_feature`, đại diện cho feature có giá trị cao nhất theo `CIFE`.

Output:
- Một giá trị số nguyên, là vị trí của feature có giá trị `CIFE` cao nhất để thêm vào solution.

In [36]:
# for generate insertion-based neighborhood of a candidate solution
def max_info_feature(candidate_solution):
    S = candidate_solution.copy()
    highest_feature = -math.inf
    highest_value = -math.inf

    for index in range(len(S)):
      if S[index] == 0:
        if CIFE[index] > highest_value:
          highest_feature = index
          highest_value = CIFE[index]
        
    return highest_feature

Hàm `swapFeature` : thực hiện thêm/bớt/hoán đổi feature bằng cách set giá trị phần tử tương ứng trong vector feature là 1 hoặc 0

Input:
- `solution`: Một mảng gồm các giá trị 0 hoặc 1, đại diện cho một tập feature được chọn.
- `i`, `j`: Các vị trí của feature cần thay đổi giá trị
- : Một số nguyên, đại diện cho vị trí của đặc trưng khác cần hoán đổi giá trị với đặc trưng tại vị trí `i`. Giá trị mặc định là -1, nếu không được cung cấp giá trị thì chỉ thay đổi giá trị của đặc trưng tại vị trí `i`.

Process:
- Nếu `j` là -1, thì đảo giá trị hiện tại của feature thứ `i` (1 - sol[i]).
- Nếu `j` khác -1, thì hoán đổi giá trị của đặc trưng tại vị trí `i` với đặc trưng tại vị trí `j`.

Output:
- Một mảng mới đại diện cho tập feature sau khi thực hiện thêm/bớt/hoán đổi feature.

In [37]:
def swapFeature(solution=None, i: int = 0, j: int = -1):
    sol = solution.copy()

    if j == -1:
        sol[i] = 1 - sol[i]
    else:
        sol[i], sol[j] = sol[j], sol[i]
        
    return sol

Hàm `initializeSolution(d)` : khởi tạo ngẫu nhiên solution ban đầu cho bài toán

Input: 
- `d`: Một số nguyên đại diện cho kích thước của vector solution ban đầu

Process:
- Hàm `np.random.randint(low=0, high=2, size=d)` được sử dụng để sinh ngẫu nhiên một mảng gồm d phần tử, mỗi phần tử có giá trị ngẫu nhiên là 0 hoặc 1.

Output:
- Một mảng gồm d phần tử, mỗi phần tử có giá trị là 0 hoặc 1, đại diện cho solution $\mathcal{S^*}$. Phần tử thứ i của mảng bằng 0 nghĩa là feature thứ i không có trong tập $\mathcal{S^*}$, và ngược lại.

In [38]:
def initializeSolution(d):
  while True:
    M = np.random.randint(low=0, high=2, size=d)
    if sum(M) < number_feature_selected_max and sum(M) > number_feature_selected_min:
      break
  return M

Hàm `getBestCandidate(S1, S2, S3)` : chọn ra tập feature có giá trị hàm mục tiêu (`F`) thấp nhất giữa ba tập `S1`, `S2`, và `S3`

Input:
- `S1`, `S2`, `S3`: Các neighbor candidates của $S^*$ được đưa vào so sánh, mỗi tập là một mảng gồm các giá trị 0 hoặc 1.

Process:
- So sánh giá trị của hàm mục tiêu (`F`) của 3 tập với nhau, `S_best` sẽ là tập có giá trị hàm `F` nhỏ nhất

Output:
- Một mảng đại diện cho tập feature tốt nhất (`S_best`) trong ba tập đầu vào `S1`, `S2`, và `S3`, dựa trên giá trị của hàm mục tiêu (`F`).

In [39]:
def getBestCandidate(S1, S2, S3):
  S_best = S1.copy()
  if F(S_best) < F(S2):
    S_best = S2.copy()
  if F(S_best) < F(S3):
    S_best = S3.copy()
  return S_best

In [40]:
ld = 1
maxTabuIterations = 50
maxLSIterations = 50

RATE_FEATURE_SELECTED_MAX = 0.3
RATE_FEATURE_SELECTED_MIN = 0.2
number_feature = X.shape[1]
number_feature_selected_max = math.ceil(number_feature * RATE_FEATURE_SELECTED_MAX)
number_feature_selected_min = math.ceil(number_feature * RATE_FEATURE_SELECTED_MIN)

In [41]:
# S = initializeSolution(X.shape[1])
# S_best = np.zeros(X.shape[1], dtype=int)
# S_star = np.zeros(X.shape[1], dtype=int)
# tabu_iterations = 0

# while tabu_iterations <= maxTabuIterations:
#     ls_iterations = 0
#     S_current = S.copy()
#     tabu_list = []
    
    
#     while ls_iterations <= maxLSIterations:
#       number_feature_current = sum(S)
#       # Generate three neighborhoods S1, S2, S3 of S that are not in tabu_list
#       index_feature_max_info = max_info_feature(S)
#       index_feature_min_info = min_info_feature(S)
#       # print(index_feature_max_info)
#       # print(index_feature_min_info)

#       if number_feature_current > number_feature_selected_min:
#         S1 = swapFeature(S, i=index_feature_min_info, j=-1)
#       if number_feature_current < number_feature_selected_max:
#         S2 = swapFeature(S, i=index_feature_max_info, j=-1)
#       if number_feature_current < number_feature_selected_max and number_feature_current > number_feature_selected_min:
#         S3 = swapFeature(S, i=index_feature_max_info, j=index_feature_min_info)

#       S_best = getBestCandidate(S1, S2, S3)
#       S = S_best

#       if F(S_best) < F(S_current):
#         S_current = S_best.copy()
        
#       # Update tabu_list
#       tabu_list.append(S_best)
      
#       ls_iterations += 1
        
#     if F(S_star) < F(S_current):
#         S_star = S_current.copy()
    
#     tabu_iterations += 1

#     print("----------------------------------------------")  
#     print("Best found Solution: {} , Objvalue: {}".format(S_current, F(S_current)))

In [49]:
tabu_iterations = 0

while tabu_iterations <= maxTabuIterations:
  S = initializeSolution(X.shape[1])
  S_best = S.copy()
  best_objective_value = F(S_best)
  S_current = S_best.copy()
  current_objective_value = F(S_current)
  print("Initial Solution: {} , Objective value: {}".format(S, F(S)))
  ls_iterations = 0
  tabu_list = {}

  while ls_iterations <= maxLSIterations:
    number_feature_current = sum(S)
    neighbor_list = {}

    # Generate three neighborhoods S1, S2, S3 of S
    index_feature_max_info = max_info_feature(S)
    index_feature_min_info = min_info_feature(S)

    if number_feature_current > number_feature_selected_min:
      neighbor_list[(index_feature_min_info, -1)] = {'objective_value' : 0}
    if number_feature_current < number_feature_selected_max:
      neighbor_list[(index_feature_max_info, -1)] = {'objective_value' : 0}
    if number_feature_current < number_feature_selected_max and number_feature_current > number_feature_selected_min:
      neighbor_list[(index_feature_max_info, index_feature_min_info)] = {'objective_value' : 0}

    for neighbor in neighbor_list:
      candidate_solution = swapFeature(S, neighbor[0], neighbor[1])
      candidate_objective_value = F(candidate_solution)
      # print("Obj move ", neighbor, "is: ", candidate_objective_value)
      neighbor_list[neighbor]['objective_value'] = candidate_objective_value

    best_move = min(neighbor_list, key=lambda x: neighbor_list[x]['objective_value'])
    best_candidate = tuple(swapFeature(S, best_move[0], best_move[1]))
    best_candidate_objective_value = neighbor_list[best_move]['objective_value']
    S = swapFeature(S_current, best_move[0], best_move[1])

    if best_candidate not in tabu_list:
      tabu_list[tuple(S_best)] = 1
      if best_candidate_objective_value < current_objective_value:
        S_current = swapFeature(S_current, best_move[0], best_move[1])
        current_objective_value = best_candidate_objective_value
        print("Update Solution: {} , Objective value: {}".format(S_current, current_objective_value))
      
    ls_iterations += 1  

  if current_objective_value < best_objective_value:
    S_best = S_current
    best_objective_value = current_objective_value
    print("Update Best Solution: {} , Objective value: {}".format(S_best, best_objective_value))
    print("++++++++++++++++++++++++++++++++++++++++++++")
      
  tabu_iterations += 1

# print("----------------------------------------------")
# print("Best found Solution: {} , Objective value: {}".format(S_best, F(S_best)))
# print("----------------------------------------------")

Initial Solution: [0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1] , Objective value: -2.241127226803627
Update Solution: [0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0] , Objective value: -3.7330038190937707
Update Solution: [0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0] , Objective value: -5.224873411383914
Update Solution: [0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0] , Objective value: -5.3124626895713725
Update Best Solution: [0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0] , Objective value: -5.3124626895713725
++++++++++++++++++++++++++++++++++++++++++++
Initial Solution: [0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0] , Objective value: -2.240968249102991
Update Solution: [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0] , Objective value: -3.732840841393152
Update Solution: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0] , Objective value: -5.2247114336833
Update Solution: [0 0 0 0 0